In [1]:
# Creating Ai Pipeline
import os
import pandas as pd
from deviceMatching.findData import *
from deviceMatching.getDict import * 
from typing import List, Tuple
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util


device_matching package loaded!


/home/anhlq/envs/matching/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading devices dictionary
from deviceMatching.getDict import * 
from deviceMatching.proccessing import load_sentence_transformer, encode_text
from sentence_transformers import SentenceTransformer, util

In [3]:
device_dictionary = load_devices_dictionary_easy()
display(device_dictionary.sample(100))

2025-02-13 15:25:20,748 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


,brand,marketing_name,model,category,year,lower_brand,lower_mktname,lower_model,no_brand_mktname,has_brand_mktname,...,non_special_has_brand_mktname,non_special_model,non_special_has_brand_model,brand_mktname_no_brand,brand_mktname_has_brand,brand_model_no_brand,brand_model_has_brand,mktname_space_count,new_marketing_name,group
19492,Samsung,UE50BU8000,UA50BU8000WXXY,TV,2022,samsung,ue50bu8000,ua50bu8000wxxy,ue50bu8000,ue50bu8000,...,ue50bu8000,ua50bu8000wxxy,ua50bu8000wxxy,samsung|ue50bu8000,samsung|ue50bu8000,samsung|ua50bu8000wxxy,samsung|ua50bu8000wxxy,0,Ue50Bu8000,DeviceSpecifications
65435,Oppo,PCRT01,PCRT01,Phone,None,oppo,pcrt01,pcrt01,pcrt01,pcrt01,...,pcrt01,pcrt01,pcrt01,oppo|pcrt01,oppo|pcrt01,oppo|pcrt01,oppo|pcrt01,0,Pcrt01,Google
11687,NEC,MultiSync E242N,MultiSync E242N,Desktop Monitor,2020,nec,multisync e242n,multisync e242n,multisync e242n,multisync e242n,...,multisync e242n,multisync e242n,multisync e242n,nec|multisynce242n,nec|multisynce242n,nec|multisynce242n,nec|multisynce242n,1,Multisync E242N,DeviceSpecifications
76192,Doro,Doro 8050,Doro 8050,Phone,None,doro,doro 8050,doro 8050,8050,doro 8050,...,doro 8050,8050,doro 8050,doro|8050,doro|doro8050,doro|8050,doro|doro8050,0,8050,Google
61075,Tvstar,TVSTAR,TVSTAR 4K Android TV,TV,None,tvstar,tvstar,tvstar 4k android tv,tvstar,tvstar,...,tvstar,4k android tv,tvstar 4k android tv,tvstar|tvstar,tvstar|tvstar,tvstar|4kandroidtv,tvstar|tvstar4kandroidtv,0,Tvstar,Google
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31864,Philips,49PFT4001/05,49PFT4001/05,TV,2016,philips,49pft4001/05,49pft4001/05,49pft4001/05,49pft4001/05,...,49pft4001 05,49pft4001 05,49pft4001 05,philips|49pft400105,philips|49pft400105,philips|49pft400105,philips|49pft400105,0,49Pft4001/05,DeviceSpecifications
21343,KingSing,T1,T1,Phone,2014,kingsing,t1,t1,t1,t1,...,t1,t1,t1,kingsing|t1,kingsing|t1,kingsing|t1,kingsing|t1,0,T1,DeviceSpecifications
51758,Haitech,HPAD_IP8045,HPAD_IP8045,Tablet,None,haitech,hpad_ip8045,hpad_ip8045,hpad ip8045,hpad ip8045,...,hpad ip8045,hpad ip8045,hpad ip8045,haitech|hpadip8045,haitech|hpadip8045,haitech|hpadip8045,haitech|hpadip8045,1,Hpad Ip8045,Google
49148,Fly,Nimbus 16,FS459,Phone,None,fly,nimbus 16,fs459,nimbus 16,nimbus 16,...,nimbus 16,fs459,fs459,fly|nimbus16,fly|nimbus16,fly|fs459,fly|fs459,1,Nimbus 16,Google


In [4]:
def precompute_column_embeddings(df: pd.DataFrame, column: str, model: SentenceTransformer, batch_size: int = 32) -> torch.Tensor:
    """
    Precompute embeddings for a specific text column in the DataFrame.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing your device dictionary.
        column (str): The name of the column to encode (e.g., "brand" or "model").
        model (SentenceTransformer): A pre-loaded SentenceTransformer model.
        batch_size (int): Number of texts to process per batch.
        
    Returns:
        torch.Tensor: A tensor of shape (num_rows, embedding_dim) containing the embeddings for the given column.
    """
    # Convert the column to strings (in case there are non-string values) and create a list.
    texts = df[column].astype(str).tolist()
    # Use the model's encode() method to compute embeddings efficiently in batches.
    embeddings = model.encode(texts, batch_size=batch_size, convert_to_tensor=True)
    return embeddings

def normalize_embeddings(embeddings: torch.Tensor) -> torch.Tensor:
    """
    Normalize each embedding vector to have unit length.
    
    Parameters:
        embeddings (torch.Tensor): A tensor with shape (N, embedding_dim).
        
    Returns:
        torch.Tensor: The normalized embeddings.
    """
    return torch.nn.functional.normalize(embeddings, p=2, dim=1)



In [61]:
#List of interested columns

columns_to_embed = ["lower_mktname", "lower_model", "lower_brand"]
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

column_embeddings = {}

for col in columns_to_embed:
    # Precompute embeddings for the current column
    embeddings = precompute_column_embeddings(device_dictionary, col, model, batch_size=32)
    # Normalize embeddings for consistent cosine similarity comparisons
    normalized = normalize_embeddings(embeddings)
    # Store the normalized embeddings in the dictionary
    column_embeddings[col] = normalized
    print(f"Computed embeddings for column '{col}' with shape: {normalized.shape}")


Computed embeddings for column 'lower_mktname' with shape: torch.Size([77141, 384])
Computed embeddings for column 'lower_model' with shape: torch.Size([77141, 384])
Computed embeddings for column 'lower_brand' with shape: torch.Size([77141, 384])


In [63]:
print("Embeddings computed for columns:", list(column_embeddings.keys()))


Embeddings computed for columns: ['lower_mktname', 'lower_model', 'lower_brand']


In [64]:
column_embeddings["lower_brand"]

tensor([[-0.0475, -0.0102,  0.0529,  ..., -0.0632,  0.0407,  0.0699],
        [-0.0475, -0.0102,  0.0529,  ..., -0.0632,  0.0407,  0.0699],
        [-0.0475, -0.0102,  0.0529,  ..., -0.0632,  0.0407,  0.0699],
        ...,
        [-0.0965,  0.0174,  0.0125,  ...,  0.0761, -0.0797,  0.0966],
        [-0.0111, -0.0045, -0.0597,  ...,  0.0854,  0.0426,  0.0461],
        [-0.0518,  0.0210, -0.0499,  ...,  0.0238,  0.0197,  0.0103]])

In [5]:
import torch
import pickle
import pandas as pd

# Define a single file for storage
save_file = "device_data.pkl"

# Prepare data to store
data_to_save = {
    "embeddings": column_embeddings,  # The precomputed embeddings
    "device_metadata": device_dictionary[["lower_mktname", "lower_model", "lower_brand"]].astype(str)  # Original device data
}

# Save everything in one file
with open(save_file, "wb") as f:
    pickle.dump(data_to_save, f)

print(f"✅ All data saved in {save_file}")


NameError: name 'column_embeddings' is not defined

In [6]:

# Load from the single file
with open("device_data.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Extract embeddings and metadata
column_embeddings = loaded_data["embeddings"]
device_metadata = loaded_data["device_metadata"]

print("✅ Loaded embeddings for columns:", list(column_embeddings.keys()))
print("✅ Loaded device metadata with shape:", device_metadata.shape)


✅ Loaded embeddings for columns: ['lower_mktname', 'lower_model', 'lower_brand']
✅ Loaded device metadata with shape: (77141, 3)


In [11]:
data_folder = "./data/"

file_path = "./data/intern/sample/log_devices_fo_app.parquet"
log_data_parquet = pd.read_parquet(file_path)


In [12]:
def log_data_clean(df):
    """
    Cleans a dataset by:
    - Replacing NaN values with "unknown".
    - Converting all text columns to lowercase, except for 'user_agent' and 'mac'.

    Parameters:
    df (pd.DataFrame): The input dataframe.

    Returns:
    pd.DataFrame: A new cleaned dataframe (original remains unchanged).
    """

    # Make a copy to avoid modifying the original dataframe1
    df_cleaned = df.copy()

    # Replace NaN values with "unknown"
    df_cleaned.fillna("unknown", inplace=True)

    # List of columns to exclude
    exclude_columns = ['user_agent', 'mac']

    # Loop through all columns and convert to lowercase if not in exclude list
    for col in df_cleaned.columns:
        if col not in exclude_columns:
            df_cleaned[col] = df_cleaned[col].astype(str).str.lower()

    return df_cleaned

log_data_parquet_processed = log_data_clean(log_data_parquet.loc[:, log_data_parquet.columns != "d"])


In [8]:
#print(embeddings_sample)
print(column_embeddings)
device_dictionary

{'lower_mktname': tensor([[-0.1020, -0.0775,  0.0728,  ...,  0.0100, -0.0149, -0.0021],
        [-0.1020, -0.0775,  0.0728,  ...,  0.0100, -0.0149, -0.0021],
        [-0.0849, -0.0770,  0.0373,  ..., -0.0438, -0.0102, -0.0059],
        ...,
        [-0.0198,  0.0074,  0.0509,  ...,  0.0182, -0.0141,  0.0045],
        [-0.0787,  0.0143, -0.0645,  ...,  0.1202,  0.0348,  0.0426],
        [-0.1153, -0.0396, -0.0588,  ...,  0.0170, -0.0437, -0.0183]]), 'lower_model': tensor([[-0.1418, -0.0198, -0.0927,  ..., -0.0193, -0.0456,  0.0114],
        [-0.1756,  0.0358, -0.1544,  ..., -0.0098, -0.0119, -0.0160],
        [-0.1742,  0.0325, -0.1421,  ..., -0.0095, -0.0300, -0.0425],
        ...,
        [-0.1160,  0.0279,  0.0253,  ...,  0.0647, -0.0994,  0.0907],
        [-0.0787,  0.0143, -0.0645,  ...,  0.1202,  0.0348,  0.0426],
        [-0.1199, -0.0462, -0.0602,  ...,  0.0107, -0.0605, -0.0278]]), 'lower_brand': tensor([[-0.0475, -0.0102,  0.0529,  ..., -0.0632,  0.0407,  0.0699],
        [-0.

,brand,marketing_name,model,category,year,lower_brand,lower_mktname,lower_model,no_brand_mktname,has_brand_mktname,...,non_special_has_brand_mktname,non_special_model,non_special_has_brand_model,brand_mktname_no_brand,brand_mktname_has_brand,brand_model_no_brand,brand_model_has_brand,mktname_space_count,new_marketing_name,group
0,Lenovo,Smart Tab M8 (3rd Gen) with Google Assistant W...,ZA8A,Tablet,2022,lenovo,smart tab m8 (3rd gen) with google assistant w...,za8a,smart tab m8 (3rd gen) with google assistant w...,smart tab m8 (3rd gen) with google assistant w...,...,smart tab m8 3rd gen with google assistant wi fi,za8a,za8a,lenovo|smarttabm83rdgenwithgoogleassistantwifi,lenovo|smarttabm83rdgenwithgoogleassistantwifi,lenovo|za8a,lenovo|za8a,8,Smart Tab M8 (3Rd Gen) With Google Assistant W...,DeviceSpecifications
1,Lenovo,Smart Tab M8 (3rd Gen) with Google Assistant W...,ZAB9,Tablet,2022,lenovo,smart tab m8 (3rd gen) with google assistant w...,zab9,smart tab m8 (3rd gen) with google assistant w...,smart tab m8 (3rd gen) with google assistant w...,...,smart tab m8 3rd gen with google assistant wi fi,zab9,zab9,lenovo|smarttabm83rdgenwithgoogleassistantwifi,lenovo|smarttabm83rdgenwithgoogleassistantwifi,lenovo|zab9,lenovo|zab9,8,Smart Tab M8 (3Rd Gen) With Google Assistant W...,DeviceSpecifications
2,Lenovo,Smart Tab M8 (3rd Gen) with Google Assistant LTE,ZA8B,Tablet,2022,lenovo,smart tab m8 (3rd gen) with google assistant lte,za8b,smart tab m8 (3rd gen) with google assistant lte,smart tab m8 (3rd gen) with google assistant lte,...,smart tab m8 3rd gen with google assistant lte,za8b,za8b,lenovo|smarttabm83rdgenwithgoogleassistantlte,lenovo|smarttabm83rdgenwithgoogleassistantlte,lenovo|za8b,lenovo|za8b,8,Smart Tab M8 (3Rd Gen) With Google Assistant Lte,DeviceSpecifications
3,Lenovo,Smart Tab M10 HD 2nd Gen Wi-Fi Google Assistant,TB-X306X,Tablet,2020,lenovo,smart tab m10 hd 2nd gen wi-fi google assistant,tb-x306x,smart tab m10 hd 2nd gen wi-fi google assistant,smart tab m10 hd 2nd gen wi-fi google assistant,...,smart tab m10 hd 2nd gen wi fi google assistant,tb x306x,tb x306x,lenovo|smarttabm10hd2ndgenwifigoogleassistant,lenovo|smarttabm10hd2ndgenwifigoogleassistant,lenovo|tbx306x,lenovo|tbx306x,8,Smart Tab M10 Hd 2Nd Gen Wi-Fi Google Assistant,DeviceSpecifications
4,Lenovo,Smart Tab M10 HD 2nd Gen Wi-Fi Google Assistant,ZA7200,Tablet,2020,lenovo,smart tab m10 hd 2nd gen wi-fi google assistant,za7200,smart tab m10 hd 2nd gen wi-fi google assistant,smart tab m10 hd 2nd gen wi-fi google assistant,...,smart tab m10 hd 2nd gen wi fi google assistant,za7200,za7200,lenovo|smarttabm10hd2ndgenwifigoogleassistant,lenovo|smarttabm10hd2ndgenwifigoogleassistant,lenovo|za7200,lenovo|za7200,8,Smart Tab M10 Hd 2Nd Gen Wi-Fi Google Assistant,DeviceSpecifications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77136,Panasonic,003P,003P,Phone,None,panasonic,003p,003p,003p,003p,...,003p,003p,003p,panasonic|003p,panasonic|003p,panasonic|003p,panasonic|003p,0,003P,Google
77137,LUVO,001L,LUVO 001L,Phone,None,luvo,001l,luvo 001l,001l,001l,...,001l,001l,luvo 001l,luvo|001l,luvo|001l,luvo|001l,luvo|luvo001l,0,001L,Google
77138,LUVO,001,Luvo 001,Phone,None,luvo,001,luvo 001,001,001,...,001,001,luvo 001,luvo|001,luvo|001,luvo|001,luvo|luvo001,0,001,Google
77139,Positivo BGH,Positivo BGH +Simple,Positivo BGH +Simple,Phone,None,positivo bgh,positivo bgh +simple,positivo bgh +simple,+simple,positivo bgh +simple,...,positivo bgh plussimple,plussimple,positivo bgh plussimple,positivobgh|plussimple,positivobgh|positivobghplussimple,positivobgh|plussimple,positivobgh|positivobghplussimple,0,+Simple,Google


In [3]:
def match_all_queries(
    embeddings_sample: torch.Tensor, 
    corpus_embeddings: torch.Tensor, 
    corpus_texts: List[str],
    sample_texts: List[str],
    threshold: float = 0.90
) -> Tuple[List[dict], List[dict]]:
    """
    Computes cosine similarity between all sample embeddings and corpus embeddings in one matrix operation.
    
    Parameters:
        embeddings_sample (torch.Tensor): Tensor of shape (N, d) for N log entry embeddings.
        corpus_embeddings (torch.Tensor): Tensor of shape (M, d) for M device dictionary embeddings.
        corpus_texts (List[str]): List of M original texts corresponding to corpus_embeddings.
        sample_texts (List[str]): List of N original log entry texts corresponding to embeddings_sample.
        threshold (float): Similarity threshold for a match.
        
    Returns:
        Tuple[List[dict], List[dict]]: Two lists containing the matching results and non-matching results.
            Each result is a dict with keys:
                "query_index": index of the sample,
                "sample_text": the original sample text,
                "matched_text": the dictionary entry that is the best match,
                "score": cosine similarity score.
    """
    # Normalize both sets of embeddings
    embeddings_sample = torch.nn.functional.normalize(embeddings_sample, p=2, dim=1)
    corpus_embeddings = torch.nn.functional.normalize(corpus_embeddings, p=2, dim=1)
    
    # Compute cosine similarity matrix (shape: N x M)
    cosine_matrix = util.cos_sim(embeddings_sample, corpus_embeddings)
    
    # For each sample, find the best matching index and corresponding score
    best_scores, best_indices = torch.max(cosine_matrix, dim=1)
    
    match_list = []
    non_match_list = []
    
    for i, (score, idx) in enumerate(zip(best_scores, best_indices)):
        result = {
            "query_index": i,
            "sample_text": sample_texts[i],
            "matched_text": corpus_texts[int(idx)],
            "score": score.item()
        }
        if score.item() >= threshold:
            match_list.append(result)
        else:
            non_match_list.append(result)
    
    return match_list, non_match_list


NameError: name 'torch' is not defined

In [13]:
print(log_data_parquet_processed)
sampled_df = log_data_parquet_processed.sample(500000).copy()

             user_id                                         user_agent  \
0         1002671200  International/1.6.2 (iPhone; iOS 16.7.10; Scal...   
1         1033612347  International/1.6.2 (iPhone; iOS 18.1.1; Scale...   
2            unknown       NGOISAO/1.5.0 (iPhone; iOS 18.1; Scale/3.00)   
3            unknown       VnExpress/7.3.5 (iPad; iOS 14.2; Scale/2.00)   
4            unknown  VnExpress/7.5.12 (iPhone; iOS 17.6.1; Scale/3.00)   
...              ...                                                ...   
20807427  1102317691                                            unknown   
20807428  1091266940                                            unknown   
20807429  1057166695                                            unknown   
20807430  1061484653                                            unknown   
20807431  1106051656                                            unknown   

            brand       model       marketing_name       os os_version  \
0           apple      ip

In [14]:
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

columns_to_embed = ["marketing_name"]

embeddings_sample = {}
for col in columns_to_embed:
    print(f"Computing embeddings for column: {col}")
    embeddings = precompute_column_embeddings(sampled_df, col, model, batch_size=1024)
    embeddings_sample[col] = embeddings
    print(f"Embeddings shape for '{col}': {embeddings.shape}")

Computing embeddings for column: marketing_name
Embeddings shape for 'marketing_name': torch.Size([500000, 384])


In [15]:
corpus_texts = device_dictionary["lower_mktname"].astype(str).tolist()
corpus_sample = sampled_df["marketing_name"].astype(str).tolist()
corpus_embeddings_dict = column_embeddings["lower_mktname"]
corpus_embeddings_sample = embeddings_sample["marketing_name"]

threshold = 0.95


In [2]:
matches, non_matches = match_all_queries(corpus_embeddings_sample, corpus_embeddings_dict, corpus_texts, corpus_sample, threshold)


NameError: name 'match_all_queries' is not defined

In [25]:
total_queries = len(matches) + len(non_matches)
num_matches = len(matches)
num_non_matches = len(non_matches)

# Count how many non-matches have the matched_text "unknown"
unknown_count = sum(1 for res in non_matches if res["sample_text"].strip().lower() == "unknown")
percentage_unknowns = (unknown_count / num_non_matches * 100) if num_non_matches > 0 else 0

print("=== Summary Information ===")
print(f"Total queries: {total_queries}")
print(f"Matches: {num_matches} ({num_matches/total_queries*100:.2f}%)")
print(f"Non-matches: {num_non_matches} ({num_non_matches/total_queries*100:.2f}%)")
print(f"Percentage of 'unknown' in non-matches: {percentage_unknowns:.2f}%")

=== Summary Information ===
Total queries: 50000
Matches: 7514 (15.03%)
Non-matches: 42486 (84.97%)
Percentage of 'unknown' in non-matches: 100.00%


In [118]:
print("=== Matches ===")
for match in matches:
    print(match)

# Print the non-matched results
print("\n=== Non-Matches ===")
for non_match in non_matches:
    print(non_match)

=== Matches ===
{'query_index': 7, 'sample_text': 'galaxy a12', 'matched_text': 'galaxy a12', 'score': 1.000000238418579}
{'query_index': 8, 'sample_text': 'galaxy s21 ultra 5g', 'matched_text': 'galaxy s21 ultra 5g', 'score': 1.0000001192092896}
{'query_index': 15, 'sample_text': 'iphone 12 pro max', 'matched_text': 'iphone 12 pro max', 'score': 1.0000001192092896}
{'query_index': 18, 'sample_text': 'a9 2020', 'matched_text': 'a9 2020', 'score': 1.0000004768371582}
{'query_index': 21, 'sample_text': 'v2040', 'matched_text': 'v2040', 'score': 1.0}
{'query_index': 29, 'sample_text': 'galaxy a51', 'matched_text': 'galaxy a51', 'score': 1.0}
{'query_index': 32, 'sample_text': 'galaxy note10+', 'matched_text': 'galaxy note10+', 'score': 1.0}
{'query_index': 37, 'sample_text': 'galaxy z fold2 5g', 'matched_text': 'galaxy z fold2 5g', 'score': 1.0000001192092896}
{'query_index': 41, 'sample_text': 'iphone 12 pro max', 'matched_text': 'iphone 12 pro max', 'score': 1.0000001192092896}
{'query_